In [52]:
# Setup and Imports
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import asyncio
from uuid import uuid4
from typing import cast

# Set environment for testing
os.environ["SKIP_DATABASE"] = "true"
os.environ["PORTKEY_BASE_URL"] = "http://localhost:8787/v1"

# Add src to path
notebook_dir = Path.cwd()
if (notebook_dir / 'src').is_dir():
    sys.path.insert(0, str(notebook_dir / 'src'))
elif (notebook_dir.parent / 'src').is_dir():
    sys.path.insert(0, str(notebook_dir.parent / 'src'))

# Import services and agents
from src.services.llm.llm_service import LLMService
from src.database.repositories import get_user_repository, get_story_repository
from src.agents.wikigen.arc_splitter import ArcSplitterAgent
from src.schemas.wikigen.arc import ArcAnalysisResult
from src.core.story_loading import StoryLoaderFactory
from dotenv import dotenv_values

print("✅ Setup complete - ready for efficient streaming tests!")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Setup complete - ready for efficient streaming tests!


In [53]:
# Initialize Services and Load Test Data

# Initialize repositories and LLM service
user_repo = get_user_repository()
story_repo = get_story_repository()
llm_service = LLMService(user_repository=user_repo)

# Load API keys from .env
env_values = dotenv_values()
AVAILABLE_PROVIDERS = {}
for provider in LLMService.get_all_llm_providers():
    provider_id = provider.provider_id
    api_key = env_values.get(f"{provider_id.upper()}_API_KEY")
    if api_key:
        AVAILABLE_PROVIDERS[provider_id] = {
            'name': provider.display_name,
            'api_key': api_key
        }

print(f"🔑 Available providers: {list(AVAILABLE_PROVIDERS.keys())}")

# Load test story
story_directory_path = Path("../tests/resources/pokemon_amber/story")
input_story = StoryLoaderFactory.load_story(story_directory_path)

# Store in repository
fake_owner_id = uuid4()
stored_story = await story_repo.store_input_story(input_story, fake_owner_id)

# Prepare story content
story_content = "\\n\\n".join([
    f"# {chapter.title}\\n{chapter.content}" 
    for chapter in input_story.chapters
])

print(f"📖 Loaded: {input_story.metadata.title}")
print(f"📊 Chapters: {len(input_story.chapters)}, Characters: {len(story_content):,}")
print(f"🎯 Ready for streaming tests!")


🔑 Available providers: ['openai', 'google', 'anthropic']
📖 Loaded: Pokemon: Ambertwo
📊 Chapters: 17, Characters: 273,713
🎯 Ready for streaming tests!


In [54]:
# Efficient Streaming Test - Single LLM Call

if not AVAILABLE_PROVIDERS:
    print("❌ No API keys available. Please add API keys to your .env file.")
else:
    # Use first available provider
    provider = list(AVAILABLE_PROVIDERS.keys())[0]
    api_key = AVAILABLE_PROVIDERS[provider]['api_key']
    model = LLMService.get_default_test_model_name_for_provider(provider)
    
    print(f"🔧 Testing with: {provider} / {model}")
    print(f"📖 Story: {input_story.metadata.title} ({len(input_story.chapters)} chapters)")
    
    # Initialize agent
    arc_splitter = ArcSplitterAgent(
        llm_service=llm_service,
        default_provider=provider,
        default_model=model
    )
    
    print(f"\\n⚡ Starting efficient streaming analysis...")
    print("📡 Live stream output:")
    print("-" * 50)
    
    # 🎯 EFFICIENT PATTERN: Single call with streaming + accumulation
    accumulated_content = ""
    chunk_count = 0
    
    async for chunk in arc_splitter.analyze_story_streaming(
        story_title=input_story.metadata.title,
        story_content=story_content[:50000],  # Truncate for demo
        total_chapters=len(input_story.chapters),
        user_id=fake_owner_id,
        api_key=api_key,
        genre="Isekai Fantasy"
    ):
        chunk_count += 1
        content = chunk.content
        accumulated_content += content  # 📊 Accumulate for parsing
        
        # ⚡ Real-time feedback to user
        if content:
            print(content, end="", flush=True)
    
    print("\\n" + "-" * 50)
    print(f"✅ Streaming completed with single LLM call!")
    print(f"📊 Chunks received: {chunk_count}")
    print(f"📝 Content length: {len(accumulated_content)} chars")
    
    # 🎯 Parse accumulated result into structured format
    print(f"\\n🔍 Parsing streamed content...")
    try:
        # Parse the accumulated JSON response
        final_result = ArcAnalysisResult.model_validate_json(accumulated_content)
        
        print(f"✅ Successfully parsed streaming result!")
        print(f"📋 Recommended arcs: {final_result.story_stats.recommended_arcs}")
        print(f"📖 Generated arcs: {len(final_result.arcs)}")
        
        # Show first arc as example
        if final_result.arcs:
            first_arc = final_result.arcs[0]
            print(f"\\n🏛️  Arc 1: {first_arc.title}")
            print(f"   📖 Chapters: {first_arc.start_chapter}-{first_arc.end_chapter}")
            print(f"   📝 Summary: {first_arc.summary[:100]}...")
            
        print(f"\\n🎯 SUCCESS: Real-time streaming + structured output with single API call!")
        
    except Exception as parse_error:
        print(f"⚠️  Parse error: {parse_error}")
        print(f"📄 Raw content: {accumulated_content[:200]}...")
        print("💡 Note: Some models may not return valid JSON in streaming mode")


🔧 Testing with: openai / gpt-4.1-nano
📖 Story: Pokemon: Ambertwo (17 chapters)
\n⚡ Starting efficient streaming analysis...
📡 Live stream output:
--------------------------------------------------
2025-06-29 01:41:16,933 - src.services.llm.llm_service - INFO - Using direct API key for provider=openai, model=gpt-4.1-nano
2025-06-29 01:41:16,962 - src.services.llm.llm_service - INFO - Model gpt-4.1-nano supports structured output - using response_format parameter
2025-06-29 01:41:16,962 - src.services.llm.llm_service - INFO - Using standard schema for provider: openai
2025-06-29 01:41:16,962 - src.services.llm.llm_service - INFO - Making LLM request: provider=openai, model=gpt-4.1-nano, gateway=http://localhost:8787/v1, streaming=True
{"arcs":[{"id":1,"title":"Introduction and Sudden Transition to a New World","start_chapter":1,"end_chapter":2,"summary":"The story begins with the protagonist immersed in a Pokémon battle on their phone, which is abruptly interrupted by a mysterious accide

# WikiGen Agent Workflow Testing

This notebook provides an environment to test the WikiGen agent-based story processing workflow using the new **BaseAgent architecture**. Each agent can be tested individually with different LLM models to demonstrate the flexibility and modularity of the system.

## 🏗️ BaseAgent Architecture

All WikiGen agents now inherit from `BaseAgent`, providing:
- **Default Models**: Each agent has configurable default provider/model
- **Override Flexibility**: Can override provider/model per call when needed  
- **Centralized LLM Logic**: Common error handling and validation
- **Clean API**: Simplified method signatures with optional parameters

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

Each agent demonstrates the BaseAgent pattern with different default models to show architectural flexibility.

## ⚙️ Setup

Make sure the Portkey Gateway is running to use the LLM Service:

```bash
docker run -d \
  --name portkey-gateway \
  -p 8787:8787 \
  portkeyai/gateway:latest
```

The following cells will:
- use the Portkey Gateway to test the LLM Service
- initialize the LLM Service
- import the WikiGen workflow agents
- load a test story from the `tests/resources/pokemon_amber/story` directory


In [55]:
# Cell 1: Setup and Configuration
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import asyncio
import logging
from uuid import uuid4
from typing import Dict, Any, List, cast

# Set environment to skip database for testing
os.environ["SKIP_DATABASE"] = "true"
os.environ["PORTKEY_BASE_URL"] = "http://localhost:8787/v1"  # Default Portkey Gateway

# Add the backend/src directory to sys.path
notebook_dir = Path.cwd()
if (notebook_dir / 'src').is_dir() and (notebook_dir / 'pyproject.toml').is_file():
    # This means we're likely in the backend/ directory itself
    sys.path.insert(0, str(notebook_dir / 'src'))
elif (notebook_dir.parent / 'src').is_dir() and (notebook_dir.parent / 'pyproject.toml').is_file():
    # This means we're likely in the backend/notebooks/ directory
    sys.path.insert(0, str(notebook_dir.parent / 'src'))
else:
    print("Warning: Could not automatically add 'src/' to Python path.")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
# Reduce noise from third-party loggers
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('uvicorn.access').setLevel(logging.WARNING)

print("🚀 WikiGen Agent Workflow Testing Environment")
print("=" * 60)
print(f"Current working directory: {os.getcwd()}")
print(f"Database mode: {'IN-MEMORY' if os.environ.get('SKIP_DATABASE') == 'true' else 'SUPABASE'}")
print(f"Portkey Gateway: {os.environ.get('PORTKEY_BASE_URL', 'Not configured')}")
print("Autoreload enabled. Changes to .py files in src/ will be reloaded.")
print("\n💡 This notebook tests WikiGen agents with real LLM calls.")
print("   Each agent can use different models to demonstrate flexibility.")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🚀 WikiGen Agent Workflow Testing Environment
Current working directory: /home/jimnix/gitrepos/shuscribe/backend/notebooks
Database mode: IN-MEMORY
Portkey Gateway: http://localhost:8787/v1
Autoreload enabled. Changes to .py files in src/ will be reloaded.

💡 This notebook tests WikiGen agents with real LLM calls.
   Each agent can use different models to demonstrate flexibility.


In [56]:
# Cell 2: Import Modules
from src.config import settings
from src.services.llm.llm_service import LLMService
from src.database.repositories import get_user_repository, get_story_repository
from src.schemas.llm.models import LLMMessage, LLMResponse
from src.core.story_loading import StoryLoaderFactory
from dotenv import dotenv_values

# WikiGen agent imports
from src.agents.wikigen import (
    WikiGenOrchestrator,
    ArcSplitterAgent,
    WikiPlannerAgent,
    ArticleWriterAgent,
    GeneralSummarizerAgent,
    ChapterBacklinkerAgent
)

print("✅ Modules imported successfully.")

# Display current settings
print("\n--- Current Settings ---")
print(f"DEBUG: {settings.DEBUG}")
print(f"ENVIRONMENT: {settings.ENVIRONMENT}")
print(f"SKIP_DATABASE: {settings.SKIP_DATABASE}")
print(f"PORTKEY_BASE_URL: {settings.PORTKEY_BASE_URL}")
print("DATABASE_MODE: In-Memory (Supabase skipped)")
print("------------------------")

print("\n🔧 Import Summary:")
print("✅ LLMService - Ready for testing")
print("✅ WikiGen Agents - All 5 agents + orchestrator imported") 
print("✅ Story Loading - Test story will be loaded")
print("✅ Repository Factory - In-memory repositories for testing")
print("✅ Pydantic Models - Type-safe schemas")


✅ Modules imported successfully.

--- Current Settings ---
DEBUG: True
ENVIRONMENT: development
SKIP_DATABASE: True
PORTKEY_BASE_URL: http://localhost:8787/v1
DATABASE_MODE: In-Memory (Supabase skipped)
------------------------

🔧 Import Summary:
✅ LLMService - Ready for testing
✅ WikiGen Agents - All 5 agents + orchestrator imported
✅ Story Loading - Test story will be loaded
✅ Repository Factory - In-memory repositories for testing
✅ Pydantic Models - Type-safe schemas


In [57]:
# Cell 3: Initialize Services and Load Test Story

print("🚀 Initializing Services and Loading Test Story...")
print("=" * 60)

# Initialize repositories and LLM service
user_repo = get_user_repository()
story_repo = get_story_repository()
llm_service = LLMService(user_repository=user_repo)

print("✅ Services initialized successfully!")
print(f"📁 Repository types: {type(user_repo).__name__}, {type(story_repo).__name__}")

# Load API keys from .env
env_values = dotenv_values()
print("\n🔑 Loading API Keys from .env...")

# Check which providers have API keys available
AVAILABLE_PROVIDERS: dict[str, dict[str, str]] = {}
for provider in LLMService.get_all_llm_providers():
    provider_id = provider.provider_id
    api_key = env_values.get(f"{provider_id.upper()}_API_KEY")
    if api_key:
        AVAILABLE_PROVIDERS[provider_id] = {
            'name': provider.display_name,
            'api_key': api_key
        }
        print(f"✅ {provider.display_name}: API key found")
    else:
        print(f"⏭️  {provider.display_name}: No API key found")

print(f"\n📊 Available Providers: {len(AVAILABLE_PROVIDERS)}")

# Load test story
print("\n📖 Loading Test Story...")
story_directory_path = Path("../tests/resources/pokemon_amber/story")

try:
    input_story = StoryLoaderFactory.load_story(story_directory_path)
    print(f"✅ Loaded: {input_story.metadata.title}")
    print(f"   Author: {input_story.metadata.author}")
    print(f"   Chapters: {input_story.total_chapters}")
    print(f"   Genres: {', '.join(input_story.metadata.genres)}")
    
    # Store in repository for testing
    fake_owner_id = uuid4()
    stored_story = await story_repo.store_input_story(input_story, fake_owner_id)
    print(f"   Stored with ID: {stored_story.id}")
    
    # Prepare story content for agent testing
    story_content = "\n\n".join([
        f"# {chapter.title}\n{chapter.content}" 
        for chapter in input_story.chapters
    ])
    
    print(f"\n📊 Test Data Summary:")
    print(f"   Total characters: {len(story_content):,}")
    print(f"   First chapter: {input_story.chapters[0].title}")
    print(f"   Last chapter: {input_story.chapters[-1].title}")
    
except Exception as e:
    print(f"❌ Error loading story: {e}")
    raise

print(f"\n🎯 Ready for WikiGen agent testing with {len(AVAILABLE_PROVIDERS)} LLM providers!")


🚀 Initializing Services and Loading Test Story...
✅ Services initialized successfully!
📁 Repository types: InMemoryUserRepository, InMemoryStoryRepository

🔑 Loading API Keys from .env...
✅ OpenAI: API key found
✅ Google: API key found
✅ Anthropic: API key found

📊 Available Providers: 3

📖 Loading Test Story...
✅ Loaded: Pokemon: Ambertwo
   Author: ChronicImmortality
   Chapters: 17
   Genres: Drama, Action, Adventure, Fantasy
   Stored with ID: 3a19f5df-e91d-4597-a021-636dbd623f3b

📊 Test Data Summary:
   Total characters: 273,664
   First chapter: [Chapter 1] Truck-kun Strikes Again
   Last chapter: [Chapter 17] The Eye of the Storm

🎯 Ready for WikiGen agent testing with 3 LLM providers!


# Run individual agents

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

## ArcSplitter Agent - Streaming Analysis

**🔄 Single LLM Call with Real-time Streaming!**

The ArcSplitter agent now supports **streaming analysis** that provides real-time feedback while accumulating the final structured result. This approach uses **only one LLM call** for both user experience and final parsing.

- ⚡ **Real-time Feedback** - See analysis progress as it happens
- 🚀 **Single LLM Call** - No wasteful duplicate API calls  
- 📊 **Live Updates** - Stream response chunks as they're generated
- 🎯 **Smart Accumulation** - Parse final accumulated result into structured data
- 💰 **Cost Efficient** - One call gives you both streaming UX and structured output


In [59]:
# Cell: 4 Test ArcSplitter Agent - Streaming Version

print("\n🔄 Testing ArcSplitter Agent - STREAMING Analysis")
print("=" * 60)
print("This demonstrates real-time streaming during story analysis.")
print("💡 You'll see the analysis progress in real-time as chunks arrive!")

if not AVAILABLE_PROVIDERS:
    print("❌ No API keys available. Please add API keys to your .env file.")
else:
    provider = "openai"  # Use Google for streaming demo    
    api_key = AVAILABLE_PROVIDERS[provider]['api_key']
    model = LLMService.get_default_test_model_name_for_provider(provider)
    
    print(f"🔧 Streaming with: {provider} / {model}")
    print(f"📊 Story: {input_story.metadata.title} ({len(input_story.chapters)} chapters)")
    
    try:
        # Initialize the agent
        arc_splitter_streaming = ArcSplitterAgent(
            llm_service=llm_service,
            default_provider=provider,
            default_model=model
        )
        
        print(f"\\n⚡ Starting streaming analysis...")
        print("📡 Live stream output:")
        print("-" * 40)
        
        # Stream the analysis in real-time and accumulate the response
        chunk_count = 0
        accumulated_content = ""
        
        async for chunk in arc_splitter_streaming.analyze_story_streaming(
            story_title=input_story.metadata.title,
            story_content=story_content,  # Truncate for faster demo
            total_chapters=len(input_story.chapters),
            user_id=fake_owner_id,
            api_key=api_key,
            genre="Isekai Fantasy"
        ):
            chunk_count += 1
            content = chunk.content
            accumulated_content += content
            
            # Print streaming content in real-time
            if content:
                print(content, end="", flush=True)
        
        print("\\n" + "-" * 40)
        print(f"✅ Streaming completed with single LLM call!")
        print(f"📊 Total chunks received: {chunk_count}")
        print(f"📝 Total content length: {len(accumulated_content)} characters")
        
        # Parse the accumulated streaming result into structured format
        print(f"\\n🎯 Parsing streamed content into structured result...")
        try:
            from src.schemas.wikigen.arc import ArcAnalysisResult
            
            # Parse the accumulated JSON response from streaming
            final_result = ArcAnalysisResult.model_validate_json(accumulated_content)
            
            print(f"✅ Successfully parsed streaming result!")
            print(f"📖 Number of arcs generated: {len(final_result.arcs)}")
            
            # Show first arc as example
            if final_result.arcs:
                first_arc = final_result.arcs[0]
                print(f"\\n🏛️  First Arc: {first_arc.title}")
                print(f"   📖 Chapters: {first_arc.start_chapter}-{first_arc.end_chapter}")
                print(f"   📝 Summary: {first_arc.summary[:100]}...")
                
        except Exception as parse_error:
            print(f"⚠️  Could not parse streaming result: {parse_error}")
            print(f"📄 Raw content preview: {accumulated_content[:200]}...")
            print("💡 This is expected if the model doesn't return valid JSON in streaming mode")
        
    except Exception as e:
        print(f"❌ Streaming test failed: {e}")
        import traceback
        traceback.print_exc()



🔄 Testing ArcSplitter Agent - STREAMING Analysis
This demonstrates real-time streaming during story analysis.
💡 You'll see the analysis progress in real-time as chunks arrive!
🔧 Streaming with: google / gemini-2.0-flash-001
📊 Story: Pokemon: Ambertwo (17 chapters)
\n⚡ Starting streaming analysis...
📡 Live stream output:
----------------------------------------
2025-06-29 01:42:42,412 - src.services.llm.llm_service - INFO - Using direct API key for provider=google, model=gemini-2.0-flash-001


2025-06-29 01:42:42,441 - src.services.llm.llm_service - INFO - Model gemini-2.0-flash-001 supports structured output - using response_format parameter
2025-06-29 01:42:42,442 - src.services.llm.llm_service - INFO - Using simplified schema for Google Gemini (removed validation constraints)
2025-06-29 01:42:42,442 - src.services.llm.llm_service - INFO - Making LLM request: provider=google, model=gemini-2.0-flash-001, gateway=http://localhost:8787/v1, streaming=True
2025-06-29 01:42:45,992 - portkey_ai._vendor.openai._base_client - INFO - Retrying request to /chat/completions in 0.461047 seconds
❌ Streaming test failed: Error code: 503 - {'error': {'message': 'Invalid response received from google: [{"error":{"code":503,"message":"The model is overloaded. Please try again later.","status":"UNAVAILABLE"}}]', 'type': None, 'param': None, 'code': None}, 'provider': 'google'}


Traceback (most recent call last):
  File "/tmp/ipykernel_159698/2464946418.py", line 34, in <module>
    async for chunk in arc_splitter_streaming.analyze_story_streaming(
  File "/home/jimnix/gitrepos/shuscribe/backend/src/agents/wikigen/arc_splitter.py", line 198, in analyze_story_streaming
    async for chunk in response_stream:
  File "/home/jimnix/gitrepos/shuscribe/backend/src/services/llm/llm_service.py", line 302, in _stream_response_to_llm_response
    else:
  File "/home/jimnix/gitrepos/shuscribe/backend/.venv/lib/python3.12/site-packages/portkey_ai/api_resources/apis/chat_complete.py", line 323, in stream_create
    async with self.openai_client.with_streaming_response.chat.completions.create(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jimnix/gitrepos/shuscribe/backend/.venv/lib/python3.12/site-packages/portkey_ai/_vendor/openai/_response.py", line 650, in __aenter__
    self.__response = await self._api_request
        

In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: General Summarizer Agent
# This utility agent creates summaries of various content types

print("\n🧪 Testing GeneralSummarizerAgent...")
print("=" * 50)

summarizer = GeneralSummarizerAgent(llm_service)

try:
    # Test summarization with a sample chapter
    sample_chapter = input_story.chapters[0]  # First chapter
    
    # Test arc summarization with sample content
    sample_arc_content = story_content[:15000]  # Use first part as arc content
    arc_metadata = {
        "title": "Test Arc",
        "start_chapter": 1,
        "end_chapter": 3
    }
    
    summary_result = await summarizer.summarize_arc(
        arc_content=sample_arc_content,
        arc_metadata=arc_metadata,
        summary_type="brief",
        user_id=TEST_USER_ID,
        provider=TEST_PROVIDER,
        model=TEST_MODEL
    )
    
    print(f"✅ GeneralSummarizerAgent completed successfully!")
    print(f"Result type: {type(summary_result)}")
    print(f"Summary result: {str(summary_result)[:500]}...")
    
except NotImplementedError as e:
    print(f"⚠️  GeneralSummarizerAgent not yet implemented: {e}")
    
except Exception as e:
    print(f"❌ GeneralSummarizerAgent failed: {e}")
    import traceback
    traceback.print_exc()


In [43]:
print(final_result.model_dump_json(indent=2))

{
  "story_stats": {
    "recommended_arcs": 4,
    "arc_strategy": "The story was divided into four arcs based on significant shifts in location, plot, and character relationships. The first arc covers Ambertwo's rebirth and escape from the lab. The second arc focuses on her exploration of Celadon City and the reveal of Dr. Fuji's Team Rocket connections. The third arc involves the side quest with the Celadon Gym. The final arc covers the climax in Pallet Town and Ambertwo's new life with Delia and Stephen."
  },
  "arcs": [
    {
      "id": 1,
      "title": "Rebirth and Escape",
      "start_chapter": 1,
      "end_chapter": 2,
      "summary": "Alexa dies after being hit by a truck while playing Pokemon Go and is reborn as Ambertwo, a clone of Dr. Fuji's deceased daughter Amber. She escapes the lab with Dr. Fuji and begins to adjust to the Pokemon world.",
      "key_events": "Character dies and is reborn as Ambertwo, Dr. Fuji reveals his intentions to recreate his dead daughter, 

## WikiPlanner Agent

In [ ]:
#
#
#

## ArticleWriterAgent

In [ ]:
#
#
#

## GeneralSummarizerAgent

In [ ]:
#
#
#

## WikiGenOrchestrator

In [ ]:
#
#
#